In [ ]:
#unzip the file im colab
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/cat_dog.zip', 'r') as zip_ref:
    zip_ref.extractall('cat_dog')

In [ ]:
#import libraires
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
# Load training and validation sets

seed=42
np.random.seed(seed)
tf.random.set_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

ds_train_ = image_dataset_from_directory(
    'cat_dog/Train',
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)
ds_valid_ = image_dataset_from_directory(
    'cat_dog/Test',
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=False,
)

# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)


In [ ]:
# build network model 
model = keras.Sequential([
    
    layers.RandomFlip(mode='horizontal'),
    layers.RandomContrast(0.1),
    layers.RandomRotation(0.1),
    layers.Conv2D(32, 3, activation='relu', input_shape=[128, 128, 3]),
    layers.MaxPool2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPool2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPool2D(),
    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid'),
])

In [ ]:
# compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

In [ ]:
# train the model
history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=50,
)

In [ ]:
#compare the model loss against the actual val_loss
history_frame = pd.DataFrame(history.history)
history_frame[['loss', 'val_loss']].plot(title="Cross-Entropy Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.show()

#compare the model accuracy against the actual val accuracy
history_frame[['binary_accuracy', 'val_binary_accuracy']].plot(title="Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid(True)
plt.show()

In [ ]:
y_ture=[]
y_pred=[]

for images, labels in ds_valid:
  pred= model.predict(images).flatten()
  y_ture.extend(labels.numpy())
  y_pred.extend(pred>0.5)


y_ture=np.array(y_ture)
y_pred=np.array(y_pred)

In [ ]:
# Confusion matrix
cm = metrics.confusion_matrix(y_ture, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Calculate performance metrics
acc = metrics.accuracy_score(y_ture, y_pred)
prec = metrics.precision_score(y_ture, y_pred)
rec = metrics.recall_score(y_ture, y_pred)
f1 = metrics.f1_score(y_ture, y_pred)

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1 Score: {f1:.4f}")

In [ ]:
# evaluate model
loss, acc = model.evaluate(ds_valid)
print(f"Test Accuracy: {acc:.4f}, Test Loss: {loss:.4f}")

In [ ]:
#save the model
model.save('Cat_Dog_after_augmentation.h5')